In [1]:
import numpy as np
import pandas as pd
import altair as alt

from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
from clickhouse_driver import Client as Clickhouse

def click_query(q, **kw):
    click = Clickhouse("localhost")
    return click.query_dataframe(q, params=kw)

In [3]:
click_query("DESCRIBE TABLE obs_web")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,probe_asn,Int32,,,,,
9,probe_cc,String,,,,,


In [72]:
click_query("DESCRIBE TABLE obs_web_ctrl")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,hostname,String,,,,,
9,observation_id,String,,,,,


In [70]:
MEASUREMENT_START_DAY = '2024-04-01'
MEASUREMENT_END_DAY = '2024-07-01'
ANALYSIS_COUNTRY_CODES = [
    "VN"
]

In [5]:
%%time
df = click_query("""
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
dns_answer_type,
http_request_url,
http_failure,
http_runtime,
http_response_body_sha1,
http_response_status_code,
http_response_body_length,
http_response_body_is_truncated,
probe_analysis
FROM obs_web
WHERE probe_cc = 'VN' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 33.3 s, sys: 3.57 s, total: 36.8 s
Wall time: 1min 43s


In [6]:
%%time
target_domains = [
    "www.bbc.com"
]
df_special = click_query("""
SELECT *
FROM obs_web 
WHERE probe_cc = 'VN' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND hostname IN %(target_domains)s
""", target_domains=target_domains)

CPU times: user 14.7 s, sys: 994 ms, total: 15.7 s
Wall time: 57 s


In [7]:
pd.set_option('display.max_colwidth', None)
df['explorer_url'] = "https://explorer.ooni.org/measurement/" + df['report_id'] + "?input=" + df['input']

In [8]:
pd.set_option('display.max_colwidth', None)
df_special['explorer_url'] = "https://explorer.ooni.org/measurement/" + df_special['report_id'] + "?input=" + df_special['input']

In [9]:
df_special.to_csv('2024-VN-df-special.csv')

In [10]:
df.to_csv('2024-VN-df.csv')

In [11]:
df = pd.read_csv('2024-VN-df.csv')

In [80]:
df[['probe_asn', 'probe_as_org_name']].drop_duplicates(subset=['probe_asn']).to_csv('2024_VN_ASN_Lookup.csv')

In [12]:
len(df)

1842437

In [13]:
df

,Unnamed: 0,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,dns_answer_type,http_request_url,http_failure,http_runtime,http_response_body_sha1,http_response_status_code,http_response_body_length,http_response_body_is_truncated,probe_analysis,explorer_url
0,0,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.facebook.com/,20240401002159.844230_VN_webconnectivity_9eb15d11cccba622,VN,18403,FPT Telecom Company,VN,wifi,2024-04-01 00:21:58,www.facebook.com,web_connectivity,163.70.159.35,443.0,32934.0,"Facebook, Inc.",118.70.115.23,VN,18403,FPT Telecom Company,VN,system,NaN,163.70.159.35,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp?input=https://www.facebook.com/
1,1,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.facebook.com/,20240401002159.844230_VN_webconnectivity_9eb15d11cccba622,VN,18403,FPT Telecom Company,VN,wifi,2024-04-01 00:21:58,www.facebook.com,web_connectivity,2a03:2880:f115:83:face:b00c:0:25de,443.0,32934.0,"Facebook, Inc.",118.70.115.23,VN,18403,FPT Telecom Company,VN,system,NaN,2a03:2880:f115:83:face:b00c:0:25de,AAAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp?input=https://www.facebook.com/
2,2,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.facebook.com/,20240401002159.844230_VN_webconnectivity_9eb15d11cccba622,VN,18403,FPT Telecom Company,VN,wifi,2024-04-01 00:21:58,www.facebook.com,web_connectivity,NaN,NaN,NaN,NaN,118.70.115.23,VN,18403,FPT Telecom Company,VN,NaN,NaN,NaN,NaN,https://www.facebook.com/,NaN,NaN,18180a17736d64d36c32c8f37ff3fd1f1bb178e4,200.0,57093.0,0.0,false,https://explorer.ooni.org/measurement/20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp?input=https://www.facebook.com/
3,3,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.instagram.com/,20240401002203.463571_VN_webconnectivity_409f176152c3ef30,VN,18403,FPT Telecom Company,VN,wifi,2024-04-01 00:22:02,www.instagram.com,web_connectivity,163.70.158.174,443.0,32934.0,"Facebook, Inc.",118.70.115.23,VN,18403,FPT Telecom Company,VN,system,NaN,163.70.158.174,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp?input=https://www.instagram.com/
4,4,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.instagram.com/,20240401002203.463571_VN_webconnectivity_409f176152c3ef30,VN,18403,FPT Telecom Company,VN,wifi,2024-04-01 00:22:02,www.instagram.com,web_connectivity,2a03:2880:f215:e8:face:b00c:0:4420,443.0,32934.0,"Facebook, Inc.",118.70.115.23,VN,18403,FPT Telecom Company,VN,system,NaN,2a03:2880:f215:e8:face:b00c:0:4420,AAAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp?input=https://www.instagram.com/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842432,1842432,20240425T083633Z_webconnectivity_VN_18403_n1_PA7NGiaiL3BgdUTa,https://www.cpj.org/,20240425094626.325923_VN_webconnectivity_c23c55cc0ef78c0f,VN,18403,FPT Telecom Company,VN,unknown,2024-04-25 09:46:25,www.cpj.org,web_connectivity,2606:4700:10::6814:50df,443.0,13335.0,Cloudflare Inc,172.253.5.4,HK,15169,Google LLC,US,system,NaN,2606:4700:10::6814:50df,AAAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240425T083633Z_webconnectivity_VN_18403_n1_PA7NGiaiL3BgdUTa?input=https://www.cpj.org/
1842433,1842433,20240425T083633Z_webconnectivity_VN_18403_n1_PA7NGiaiL3BgdUTa,https://www.cpj.org

In [14]:
df_special

,measurement_uid,input,report_id,measurement_start_time,software_name,software_version,test_name,test_version,probe_asn,probe_cc,probe_as_org_name,probe_as_cc,probe_as_name,network_type,platform,origin,engine_name,engine_version,architecture,resolver_ip,resolver_asn,resolver_cc,resolver_as_org_name,resolver_as_cc,resolver_is_scrubbed,resolver_asn_probe,resolver_as_org_name_probe,bucket_date,observation_id,created_at,post_processed_at,target_id,hostname,transaction_id,ip,port,ip_asn,ip_as_org_name,ip_as_cc,ip_cc,ip_is_bogon,dns_query_type,dns_failure,dns_engine,dns_engine_resolver_address,dns_answer_type,dns_answer,dns_answer_asn,dns_answer_as_org_name,dns_t,tcp_failure,tcp_success,tcp_t,tls_failure,tls_server_name,tls_version,tls_cipher_suite,tls_is_certificate_valid,tls_end_entity_certificate_fingerprint,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_certificate_chain_fingerprints,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_handshake_time,tls_t,http_request_url,http_network,http_alpn,http_failure,http_request_body_length,http_request_method,http_runtime,http_response_body_length,http_response_body_is_truncated,http_response_body_sha1,http_response_status_code,http_response_header_location,http_response_header_server,http_request_redirect_from,http_request_body_is_truncated,http_t,probe_analysis,pp_http_response_fingerprints,pp_http_fingerprint_country_consistent,pp_http_response_matches_blockpage,pp_http_response_matches_false_positive,pp_http_response_body_title,pp_http_response_body_meta_title,pp_dns_fingerprint_id,pp_dns_fingerprint_country_consistent,explorer_url
0,20240110002215.404525_VN_webconnectivity_1b66adcc6f9f8a8d,https://www.bbc.com/,20240110T001108Z_webconnectivity_VN_45899_n1_6sC9NC1erlwHaXWa,2024-01-10 00:22:14,ooniprobe-android-unattended,3.8.5.1,web_connectivity,0.4.3,45899,VN,Vietnam Posts and Telecommunications (VNPT),VN,20171119,wifi,android,autorun,ooniprobe-engine,3.19.1,arm64,113.164.250.3,45899,VN,Vietnam Posts and Telecommunications (VNPT),VN,0,45899,Vietnam Posts and Telecommunications (VNPT),2024-01-10,20240110002215.404525_VN_webconnectivity_1b66adcc6f9f8a8d_0,2024-01-11 03:14:01,None,None,www.bbc.com,None,151.101.76.81,443.0,54113.0,"Fastly, Inc.",US,HK,0.0,A,None,system,,A,151.101.76.81,54113.0,"Fastly, Inc.",5.029923,None,1.0,6.518904,generic_timeout_error,www.bbc.com,,,NaN,None,None,None,None,None,[],NaT,NaT,NaN,[],0.0,1.0,0.0,277.0,write_1,10.001790,16.520695,None,None,None,None,NaN,None,None,NaN,NaN,None,NaN,None,None,None,NaN,NaN,http-failure,[],None,0,0,None,None,None,None,https://explorer.ooni.org/measurement/20240110T001108Z_webconnectivity_VN_45899_n1_6sC9NC1erlwHaXWa?input=https://www.bbc.com/
1,20240110002215.404525_VN_webconnectivity_1b66adcc6f9f8a8d,https://www.bbc.com/,20240110T001108Z_webconnectivity_VN_45899_n1_6sC9NC1erlwHaXWa,2024-01-10 00:22:14,ooniprobe-android-unattended,3.8.5.1,web_connectivity,0.4.3,45899,VN,Vietnam Posts and Telecommunications (VNPT),VN,20171119,wifi,android,autorun,ooniprobe-engine,3.19.1,arm64,113.164.250.3,45899,VN,Vietnam Posts and Telecommunications (VNPT),VN,0,45899,Vietnam Posts and Telecommunications (VNPT),2024-01-10,20240110002215.404525_VN_webconnectivity_1b66adcc6f9f8a8d_1,2024-01-11 03:14:01,None,None,www.bbc.com,None,None,NaN,NaN,None,None,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,[],NaT,NaT,NaN,[],NaN,NaN,NaN,NaN,None,NaN,NaN,https://www.bbc.com/,tcp,None,generic_timeout_error,0.0,GET,None,NaN,0.0,None,0.0,None,None,None,0.0,26.599899,http-failure,[],None,0,0,None,None,None,None,https://explorer.ooni.org/measuremen

In [15]:
df['probe_analysis'].value_counts()

false           1718176
http-failure      49773
dns               18763
tcp_ip            15794
http-diff          2226
Name: probe_analysis, dtype: int64

In [16]:
df_dns_answer = df[df['dns_answer_type']=='A'].drop_duplicates(subset=['measurement_uid'])

In [17]:
len(df_dns_answer)

446489

In [18]:
dns_resp_sorted = df_dns_answer[
    (df_dns_answer['probe_analysis'] == 'dns')
].groupby('dns_answer')['hostname'].nunique().sort_values().reset_index()

In [19]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 2
]

,dns_answer,hostname
93,162.210.196.167,3
94,162.210.196.166,3
95,69.162.95.2,3
96,192.157.56.141,3
97,192.157.56.140,3
98,199.115.116.216,3
99,207.244.65.58,3
100,74.63.241.23,3
101,37.48.65.153,3
102,69.162.95.6,3


In [20]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 10
]

,dns_answer,hostname
109,103.199.79.14,25
110,125.235.4.59,31
111,127.0.0.1,143


In [21]:
dns_suspicious = dns_resp_sorted[dns_resp_sorted['hostname'] > 10]['dns_answer'].values.tolist()
dns_suspicious


['103.199.79.14', '125.235.4.59', '127.0.0.1']

In [22]:
df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)].drop_duplicates(subset=['dns_answer']).to_csv('2024_VN_dns_suspicious.csv')

In [23]:
dns_answer_filter = df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)]

dns_answer_filter = dns_answer_filter.pivot_table(index=['dns_answer', 'probe_cc'], 
              values=['hostname'],
              aggfunc={'hostname': ['nunique', lambda x: ', '.join(set(x))]}).rename(columns = {
                                                                    '<lambda_0>': 'hostnames',
                                                                    'len' : 'hostname_count'}).reset_index()

dns_answer_filter.head()

dns_answer probe_cc  \
                            
0  103.199.79.14       VN   
1   125.235.4.59       VN   
2      127.0.0.1       VN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     hostname  \
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [24]:
dns_answer_filter.to_csv('2024_VN_dns_answer_hostname')

In [25]:
df_http_answer = df[df['http_request_url'].notnull()].drop_duplicates(subset=['measurement_uid'])

In [26]:
http_domain_count = df_http_answer[
    df_http_answer['probe_analysis'] == 'http-diff'
].groupby('http_response_body_sha1')['hostname'].nunique().sort_values().reset_index()

In [27]:
http_domain_count[
    http_domain_count['hostname'] > 2
]

,http_response_body_sha1,hostname
391,76e47a8682455d2e891b901bba7d93e5f379454a,4
392,84b230b3974fccbcf327a7101aac4a5eee5145f9,21


In [28]:
http_domain_count[
    http_domain_count['hostname'] > 10
]

,http_response_body_sha1,hostname
392,84b230b3974fccbcf327a7101aac4a5eee5145f9,21


In [29]:
http_suspicious = http_domain_count[http_domain_count['hostname'] > 2]['http_response_body_sha1'].values.tolist()
http_suspicious


['76e47a8682455d2e891b901bba7d93e5f379454a',
 '84b230b3974fccbcf327a7101aac4a5eee5145f9']

In [30]:
df_http_answer[df_http_answer['http_response_body_sha1'].isin(http_suspicious)].drop_duplicates(subset=['http_response_body_sha1']).to_csv('2024_VN_http_suspicious.csv')

In [69]:
confirmed_ips = [
    '103.192.236.94'
]

likely_ips = [
    '125.235.4.59',
    '125.212.193.174',
    '103.199.79.14']

false_positive_ips = [
    '45.77.240.186'
]

confirmed_http_resp = [
    '84b230b3974fccbcf327a7101aac4a5eee5145f9'
]

likely_http_resp = [
]

false_positive_http_resp = [
]


In [71]:
%%time
df_with_failure = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_certificate_chain_length,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'VN' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 1min 6s, sys: 5.63 s, total: 1min 12s
Wall time: 2min 10s


In [72]:
pd.set_option('display.max_colwidth', None)
df_with_failure['explorer_url'] = "https://explorer.ooni.org/m/" + df_with_failure['measurement_uid']

In [73]:
df_with_failure["probe_asn"] = "AS" + df_with_failure["probe_asn"].apply(str)

In [74]:
df_with_failure.head()

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_response_body_sha1,http_request_url,http_failure,failure_class,failure_str_full,failure_str,failure_str_raw,explorer_url
0,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.facebook.com/,20240401002159.844230_VN_webconnectivity_9eb15d11cccba622,VN,AS18403,FPT Telecom Company,VN,wifi,2024-04-01 00:21:58,www.facebook.com,web_connectivity,163.70.159.35,443.0,32934.0,"Facebook, Inc.",118.70.115.23,VN,18403,FPT Telecom Company,VN,system,None,163.70.159.35,1.0,None,0.033927,3.0,4.0,3245.0,394.0,write_4,1,1.0,"CN=*.facebook.com,O=Meta Platforms\, Inc.,L=Menlo Park,ST=California,C=US",*.facebook.com,"CN=DigiCert SHA2 High Assurance Server CA,OU=www.digicert.com,O=DigiCert Inc,C=US",DigiCert SHA2 High Assurance Server CA,"[*.facebook.com, *.facebook.net, *.fbcdn.net, *.fbsbx.com, *.m.facebook.com, *.messenger.com, *.xx.fbcdn.net, *.xy.fbcdn.net, *.xz.fbcdn.net, facebook.com, messenger.com]",2024-04-08 23:59:59,2024-01-09,2.0,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240401002159.844230_VN_webconnectivity_9eb15d11cccba622
1,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.facebook.com/,20240401002159.844230_VN_webconnectivity_9eb15d11cccba622,VN,AS18403,FPT Telecom Company,VN,wifi,2024-04-01 00:21:58,www.facebook.com,web_connectivity,2a03:2880:f115:83:face:b00c:0:25de,443.0,32934.0,"Facebook, Inc.",118.70.115.23,VN,18403,FPT Telecom Company,VN,system,None,2a03:2880:f115:83:face:b00c:0:25de,1.0,None,0.031570,3.0,4.0,3243.0,394.0,write_4,1,1.0,"CN=*.facebook.com,O=Meta Platforms\, Inc.,L=Menlo Park,ST=California,C=US",*.facebook.com,"CN=DigiCert SHA2 High Assurance Server CA,OU=www.digicert.com,O=DigiCert Inc,C=US",DigiCert SHA2 High Assurance Server CA,"[*.facebook.com, *.facebook.net, *.fbcdn.net, *.fbsbx.com, *.m.facebook.com, *.messenger.com, *.xx.fbcdn.net, *.xy.fbcdn.net, *.xz.fbcdn.net, facebook.com, messenger.com]",2024-04-08 23:59:59,2024-01-09,2.0,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240401002159.844230_VN_webconnectivity_9eb15d11cccba622
2,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.facebook.com/,20240401002159.844230_VN_webconnectivity_9eb15d11cccba622,VN,AS18403,FPT Telecom Company,VN,wifi,2024-04-01 00:21:58,www.facebook.com,web_connectivity,None,NaN,NaN,None,118.70.115.23,VN,18403,FPT Telecom Company,VN,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,18180a17736d64d36c32c8f37ff3fd1f1bb178e4,https://www.facebook.com/,None,ok,ok,,,https://explorer.ooni.org/m/20240401002159.844230_VN_webconnectivity_9eb15d11cccba622
3,20240401T002157Z_webconnectivity_VN_18403_n1_PdwxneoVqRU4pPJp,https://www.instagram.com/,20240401002203.463571_VN_webconnectivity_409f176152c3ef30,VN,AS18403,FPT Telecom Company,VN,wifi,2024-04-01 00:22:02,www.instagram.com,web_connectivity,163.70.158.174,443.0,32934.0,"Facebook, Inc.",118.70.115.23,VN,18403,FPT Telecom Company,VN,system,None,163.70.158.174,1.0,None,0.040478,3.0,4.0,3503.0,395.0,write_4,1,1.0,"CN=*.www.instagram.com,O=Meta Platforms\, Inc.,L=Menlo Park,ST=California,C=US",*.www.instagram.com,"CN=DigiCert SHA2 High Assurance Server CA,OU

In [75]:
df_with_failure.to_csv('2024-VN-df_with_failure.csv')

In [37]:
#df_with_failure = pd.read_csv('2024-VN-df_with_failure.csv')

In [76]:
len(df_with_failure)

1842437

In [77]:
list(df_with_failure)

['report_id',
 'input',
 'measurement_uid',
 'probe_cc',
 'probe_asn',
 'probe_as_org_name',
 'probe_as_cc',
 'network_type',
 'measurement_start_time',
 'hostname',
 'test_name',
 'ip',
 'port',
 'ip_asn',
 'ip_as_org_name',
 'resolver_ip',
 'resolver_cc',
 'resolver_asn',
 'resolver_as_org_name',
 'resolver_as_cc',
 'dns_engine',
 'dns_failure',
 'dns_answer',
 'tcp_success',
 'tcp_failure',
 'tls_handshake_time',
 'tls_handshake_read_count',
 'tls_handshake_write_count',
 'tls_handshake_read_bytes',
 'tls_handshake_write_bytes',
 'tls_handshake_last_operation',
 'tls_success',
 'tls_is_certificate_valid',
 'tls_end_entity_certificate_subject',
 'tls_end_entity_certificate_subject_common_name',
 'tls_end_entity_certificate_issuer',
 'tls_end_entity_certificate_issuer_common_name',
 'tls_end_entity_certificate_san_list',
 'tls_end_entity_certificate_not_valid_after',
 'tls_end_entity_certificate_not_valid_before',
 'tls_certificate_chain_length',
 'tls_failure',
 'http_response_body_s

In [78]:
unknown_failure_map = {
    ': server misbehaving': 'dns_server_misbehaving',
    ': read: connection refused': 'connection_refused',
    ': connect: network is unreachable': 'network_unreachable',
    'tls: first record does not look like a TLS handshake': 'tls_bad_first_record',
    'remote error: tls: handshake failure': 'tls_handshake_failure',
    'remote error: tls: illegal parameter': 'tls_illegal_parameter',
    'connectex: No connection could be made because the target machine actively refused it': 'connection_refused',
    'read: connection refused': 'connection_refused',
    'remote error: tls: access denied': 'tls_access_denied',
    'remote error: tls: internal error': 'tls_internal_error',
    'HTTP/1.x transport connection broken: malformed HTTP version': 'http_malformed_response',
    'net/http: timeout awaiting response headers': 'http_timeout',
    'read: operation timed out': 'timed_out',
    'connect: operation timed out': 'timed_out',
    ': No address associated with hostname': 'dns_nxdomain_error',
    ': connect: bad file descriptor': 'bad_file_descriptor',
    'stream error: stream ID': 'http_stream_error',
    'readLoopPeekFailLocked: <nil>': 'http_golang_bug',
    ': connect: no route to host': 'host_unreachable',
    ': connect: cannot assign requested address': 'address_not_available',
    'getaddrinfow: The requested name is valid, but no data of the requested type was found.': 'dns_no_answer',
    'wsarecv: Se ha forzado la interrupción de una conexión existente por el host remoto.': 'connection_reset',
    'wsarecv: An existing connection was forcibly closed by the remote host.': 'connection_reset',
    'wsarecv: Connessione in corso interrotta forzatamente dall\'host remoto.': 'connection_reset',
    'wsarecv: Uma ligação existente foi forçada a fechar pelo anfitrião remoto': 'connection_reset',
    'getaddrinfow: Ceci est habituellement une erreur temporaire qui se produit durant la résolution du nom d’hôte et qui signifie que le serveur local n’a pas reçu de réponse d’un serveur faisant autorité': 'dns_temporary_failure',
    'getaddrinfow: Dies ist normalerweise ein zeitweiliger Fehler bei der Auflösung von Hostnamen. Grund ist, dass der lokale Server keine Rückmeldung vom autorisierenden Server erhalten hat.': 'dns_temporary_failure',
    'getaddrinfow: Este é geralmente um erro temporário durante a resolução de nomes de anfitrião e significa que o servidor local não recebeu uma resposta de um servidor autoritário': 'dns_temporary_failure',
    'getaddrinfow: Éste es normalmente un error temporal durante la resolución de nombres de host y significa que el servidor local no recibió una respuesta de un servidor autoritativo': 'dns_temporary_failure',
}

def map_unknown_failure(failure_str):
    if pd.isna(failure_str):  # Handle NaN values
        return "unknown_failure_nan"  # or return a default value

    if not isinstance(failure_str, str):  # Ensure it's a string
        failure_str = str(failure_str)

    if not failure_str.startswith("unknown_failure"):
        return failure_str

    for substring, clean_failure in unknown_failure_map.items():
        if substring in failure_str:
            return clean_failure

    return "unknown_failure"


In [79]:
def simplify_failure(failure_str):
    if failure_str in ['timed_out', 'generic_timeout_error', 'deferred_timeout_error']:
        return 'timeout'
    
    if failure_str in ['android_dns_cache_no_data', 'dns_nxdomain_error']:
        return 'nxdomain'
    
    if failure_str in ['connection_refused', 'connection_refused_error']:
        return 'connection_refused'
        
    return failure_str

ipv6_failures = ['address_not_available', 'address_family_not_supported', 'network_unreachable', 'host_unreachable']

In [80]:
def compute_analysis(row):
    failure_str = map_unknown_failure(row['failure_str_raw'])

    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in confirmed_ips:
        return 'dns.confirmed'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in likely_ips:
        return 'dns.likely.blocked'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in false_positive_ips:
        return 'ok'
        
    if row['tls_is_certificate_valid'] == True:
        return 'ok'

    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in confirmed_http_resp:
        return 'http.confirmed'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in likely_http_resp:
        return 'http.likely.blocked'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in false_positive_http_resp:
        return 'ok'
    
    if row['failure_class'] == 'ok':
        return 'ok'

    if row['dns_consistency'] == 'inconsistent':
        return 'dns.inconsistent'
    
    if row['ip_as_org_name'] == 'Bogon':
        return 'dns.bogon'
    
    if pd.isnull(row['ip']):  # Check if ip is NaN
        return 'unknown_error'
    
    simple_failure = simplify_failure(failure_str)
    
    if simple_failure in ipv6_failures and ':' in row['ip']:  # Check ':' in row['ip'] only if row['ip'] is not NaN
        return 'ipv6_error'

    if simple_failure.startswith("ssl_"):
        simple_failure = 'bad_cert'
    
    prefix = row['failure_class']
    if prefix == 'https' and simple_failure.startswith('dns_') or simple_failure == 'nxdomain':
        prefix = 'dns'

    return f'{prefix}.{simple_failure}'


old code

In [130]:
# see: https://learn.microsoft.com/en-us/windows/win32/winsock/windows-sockets-error-codes-2
unknown_failure_map = {
    ': server misbehaving': 'dns_server_misbehaving',
    ': read: connection refused': 'connection_refused',
    ': connect: network is unreachable': 'network_unreachable',
    'tls: first record does not look like a TLS handshake': 'tls_bad_first_record',
    'remote error: tls: handshake failure': 'tls_handshake_failure',
    'remote error: tls: illegal parameter': 'tls_illegal_parameter',
    'connectex: No connection could be made because the target machine actively refused it': 'connection_refused',
    'read: connection refused': 'connection_refused',
    'remote error: tls: access denied': 'tls_access_denied',
    'remote error: tls: internal error': 'tls_internal_error',
    'HTTP/1.x transport connection broken: malformed HTTP version': 'http_malformed_response',
    'net/http: timeout awaiting response headers': 'http_timeout',
    'read: operation timed out': 'timed_out',
    'connect: operation timed out': 'timed_out',
    ': No address associated with hostname': 'dns_nxdomain_error',
    ': connect: bad file descriptor': 'bad_file_descriptor',
    'stream error: stream ID': 'http_stream_error',
    
    # This looks more like a golang-bug: https://github.com/golang/go/issues/31259
    'readLoopPeekFailLocked: <nil>': 'http_golang_bug',

    ': connect: no route to host': 'host_unreachable',
    ': connect: cannot assign requested address': 'address_not_available',
    'getaddrinfow: The requested name is valid, but no data of the requested type was found.': 'dns_no_answer',
    'wsarecv: Se ha forzado la interrupción de una conexión existente por el host remoto.': 'connection_reset',
    'wsarecv: An existing connection was forcibly closed by the remote host.': 'connection_reset',
    'wsarecv: Connessione in corso interrotta forzatamente dall\'host remoto.': 'connection_reset',
    'wsarecv: Uma ligação existente foi forçada a fechar pelo anfitrião remoto': 'connection_reset',
    
    'getaddrinfow: Ceci est habituellement une erreur temporaire qui se produit durant la résolution du nom d’hôte et qui signifie que le serveur local n’a pas reçu de réponse d’un serveur faisant autorité': 'dns_temporary_failure',
    'getaddrinfow: Dies ist normalerweise ein zeitweiliger Fehler bei der Auflösung von Hostnamen. Grund ist, dass der lokale Server keine Rückmeldung vom autorisierenden Server erhalten hat.': 'dns_temporary_failure',
    'getaddrinfow: Este é geralmente um erro temporário durante a resolução de nomes de anfitrião e significa que o servidor local não recebeu uma resposta de um servidor autoritário': 'dns_temporary_failure',
    'getaddrinfow: Éste es normalmente un error temporal durante la resolución de nombres de host y significa que el servidor local no recibió una respuesta de un servidor autoritativo': 'dns_temporary_failure',
}
def map_unknown_failure(failure_str):
    if not failure_str.startswith("unknown_failure"):
        return failure_str
    for substring, clean_failure in unknown_failure_map.items():
        if substring in failure_str:
            return clean_failure
    return "unknown_failure"

def simplify_failure(failure_str):
    if failure_str in ['timed_out', 'generic_timeout_error', 'deferred_timeout_error']:
        return 'timeout'
    
    if failure_str in ['android_dns_cache_no_data', 'dns_nxdomain_error']:
        return 'nxdomain'
    
    if failure_str in ['connection_refused', 'connection_refused_error']:
        return 'connection_refused'
        
    return failure_str

ipv6_failures = ['address_not_available', 'address_family_not_supported', 'network_unreachable', 'host_unreachable']
def compute_analysis(row):
    failure_str = map_unknown_failure(row['failure_str_raw'])

    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in confirmed_ips:
        return 'dns.confirmed'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in likely_ips:
        return 'dns.likely.blocked'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in false_positive_ips:
        return 'ok'
        
    if row['tls_is_certificate_valid'] == True:
        return 'ok'

    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in confirmed_http_resp:
        return 'http.confirmed'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in likely_http_resp:
        return 'http.likely.blocked'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in false_positive_http_resp:
        return 'ok'
    
    if row['failure_class'] == 'ok':
        return 'ok'

    if row['dns_consistency'] == 'inconsistent':
        return 'dns.inconsistent'
    
    if row['ip_as_org_name'] == 'Bogon':
        return 'dns.bogon'
    
    #if row['dns_blocking_scope'] not in ('u', 'n'):
    #    return f"dns.{row['dns_blocking_scope']}"
    
    if row['tls_is_certificate_valid'] == False:
        return 'tls.bad_cert'
    
    simple_failure = simplify_failure(failure_str)
    if simple_failure in ipv6_failures and row['ip'] and ':' in row['ip']:
        return 'ipv6_error'

    if simple_failure.startswith("ssl_"):
        simple_failure = 'bad_cert'
    
    prefix = row['failure_class']
    if prefix == 'https' and simple_failure.startswith('dns_') or simple_failure == 'nxdomain':
        prefix = 'dns'

    return f'{prefix}.{simple_failure}'

In [81]:
df_ctrl_dns_obs_web = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web
WHERE measurement_start_time > %(measurement_start_day)s
AND probe_cc NOT IN %(cc_list)s
AND tls_is_certificate_valid = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY,
    "cc_list": ANALYSIS_COUNTRY_CODES,
})

In [82]:
%%time
df_ctrl_dns_obs_web_ctrl = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web_ctrl
WHERE measurement_start_time > %(measurement_start_day)s
AND tls_success = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY
})

CPU times: user 1.14 s, sys: 134 ms, total: 1.27 s
Wall time: 1min 6s


In [83]:
from collections import defaultdict
ctrl_map = defaultdict(dict)
for _, row in df_ctrl_dns_obs_web_ctrl.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_ctrl'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_ctrl'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_ctrl'] = set(row['uniq_as_org_name'])
    
for _, row in df_ctrl_dns_obs_web.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_web'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_web'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_web'] = set(row['uniq_as_org_name'])

In [84]:
df_ctrl_dns_obs_web.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,[2a06:98c1:3121::3],[13335],[Cloudflare Inc],desk.im
1,"[46.107.16.55, 46.107.16.54, 2001:4c48:16:6::1:20, 2001:4c48:16:6::2:20]",[5483],[Magyar Telekom plc.],videa.hu
2,[185.34.219.59],[198203],[RouteLabel V.O.F.],women.jo
3,"[172.67.179.43, 104.21.31.185]",[13335],[Cloudflare Inc],es.zenit.org
4,"[2606:50c0:8002::153, 185.199.110.153, 185.199.109.153, 2606:50c0:8001::153, 185.199.108.153, 185.199.111.153, 2606:50c0:8000::153, 2606:50c0:8003::153]",[54113],"[Fastly, Inc.]",sibrea.github.io


In [85]:
df_ctrl_dns_obs_web_ctrl.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,"[188.114.97.0, 2606:4700:3036::ac43:a8d1, 188.114.96.3, 172.67.168.209, 2a06:98c1:3121::9, 2606:4700:3036::6815:2eb2, 188.114.97.4, 2a06:98c1:3121::3, 104.21.46.178, 2a06:98c1:3121::, 188.114.96.9, 2a06:98c1:3120::4, 2a06:98c1:3121::4, 188.114.97.9, 2a06:98c1:3120::, 2a06:98c1:3120::3, 188.114.96.4, 2a06:98c1:3120::9, 188.114.97.3, 188.114.96.0]",[13335],[Cloudflare Inc],ishr.ch
1,"[104.21.24.126, 2606:4700:3032::ac43:dacb, 172.67.218.203, 2606:4700:3031::6815:187e]",[13335],[Cloudflare Inc],ppy.buzz
2,[31.133.222.2],[50245],[Serverel Inc.],fpo.xxx
3,"[23.50.131.148, 2a02:26f0:c900:b::5f65:4a03, 23.53.43.17, 2a02:26f0:b200::58dd:1933, 95.101.111.141, 23.50.131.154, 2.20.143.81, 23.36.162.196, 2a02:26f0:3500:14::1724:a249, 2.16.6.11, 2a02:26f0:ab00::5c7a:d728, 2.18.244.74, 2a02:26f0:b200::58dd:193a, 2.20.143.97, 23.36.162.73, 95.101.111.157, 2a02:26f0:ab00::214:8e4a, 2a02:26f0:c900:b::5f65:4a18, 2.18.244.96, 23.53.43.74, 2a02:26f0:3500:14::1724:a258, 104.110.191.13, 23.36.162.216, 2a02:26f0:1180:33::210:64c, 104.110.191.46, 2.16.6.23, 2a02:26f0:1180:33::210:64f, 23.36.162.88]",[20940],[Akamai International B.V.],trouw.nl
4,"[104.16.41.196, 13.49.232.30, 2606:4700::6810:2cc4, 2606:4700::6810:2ac4, 2606:4700::6810:2dc4, 104.16.45.196, 104.16.42.196, 104.16.43.196, 13.48.174.154, 2606:4700::6810:2bc4, 104.16.44.196, 2606:4700::6810:29c4]","[13335, 16509]","[Cloudflare Inc, Amazon.com, Inc.]",www.redporn.com


In [121]:
def compute_dns_consistency(row):
    if pd.isnull(row['ip']):
        return None
    ctrl = ctrl_map[row['hostname']]
    if len(ctrl):
        return 'unknown'
    if row['ip'] in ctrl.get('uniq_ip_ctrl', {}):        
        return 'consistent.ip_ctrl'
    if row['ip'] in ctrl.get('uniq_ip_web', {}):
        return 'consistent.ip_web'
    if row['ip_asn'] in ctrl.get('uniq_asn_ctrl', {}):
        return 'consistent.asn_ctrl'
    if row['ip_asn'] in ctrl.get('uniq_asn_web', {}):
        return 'consistent.asn_web'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_ctrl', {}):
        return 'consistent.as_org_name_ctrl'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_web', {}):
        return 'consistent.as_org_name_web'
    return 'inconsistent'

In [131]:
df_with_failure['dns_consistency'] = df_with_failure.progress_apply(compute_dns_consistency, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3766/3766 [00:00<00:00, 68656.87it/s]


In [132]:
df_with_failure['analysis'] = df_with_failure.progress_apply(compute_analysis, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3766/3766 [00:00<00:00, 27097.26it/s]


In [91]:
confirmed_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.confirmed') | 
    (df_with_failure['analysis'] == 'http.confirmed')]

msmt_counts_confirmed = confirmed_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': len, 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [92]:
likely_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.likely.blocked') |
    (df_with_failure['analysis'] == 'http.likely.blocked')]

msmt_counts_likely = likely_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': len, 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [93]:
msmt_counts_confirmed.to_csv('2024-vietnam-confirmed-blocked-heuristics.csv')

In [94]:
msmt_counts_likely.to_csv('2024-vietnam-likely-blocked-heuristics.csv')

In [95]:
df_with_failure[
     (df_with_failure['analysis'] != 'ok') & (df_with_failure['hostname'] == "www.bbc.com")
]['analysis'].unique()

array(['tls.connection_reset', 'unknown_error', 'dns.bogon',
       'tls.timeout', 'tls.eof_error', 'tcp.timeout'], dtype=object)

In [138]:
%%time
target_domains = [
    "www.bbc.com"
]
df_special = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_certificate_chain_length,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'VN' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
AND hostname IN %(target_domains)s
""", target_domains=target_domains)

CPU times: user 10.3 s, sys: 801 ms, total: 11.1 s
Wall time: 1min 29s


In [139]:
len(df_special)

28714

In [140]:
df_special['dns_consistency'] = df_special.progress_apply(compute_dns_consistency, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28714/28714 [00:00<00:00, 116645.55it/s]


In [141]:
df_special['analysis'] = df_special.progress_apply(compute_analysis, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28714/28714 [00:00<00:00, 34067.26it/s]


In [142]:
df_special["probe_asn"] = "AS" + df_special["probe_asn"].apply(str)

In [162]:
method_of_blocking = df_special[df_special['analysis'] != 'ok'].pivot_table(index=['analysis'], values=['measurement_uid'], columns=['probe_asn','probe_as_org_name'],
                                                aggfunc={'measurement_uid': len}).fillna(0).reset_index()

In [163]:
method_of_blocking

analysis      measurement_uid  \
probe_asn                                               AS131429   
probe_as_org_name                           MOBIFONE Corporation   
0                                 dns.bogon                  0.0   
1                         dns.dns_no_answer                  0.0   
2                        dns.likely.blocked                 20.0   
3                              dns.nxdomain                  0.0   
4                               dns.timeout                  0.0   
5                            http.confirmed                  0.0   
6                  https.connection_refused                  0.0   
7                    https.connection_reset                  0.0   
8                           https.eof_error                  0.0   
9                             https.timeout                 20.0   
10                               ipv6_error                 20.0   
11                              tcp.timeout                  0.0   
12                     tls.connection_reset                  0.0   
13                            tls.eof_error                  0.0   
14                              tls.timeout                  0.0   

                                       \
probe_asn                    AS133606   
probe_as_org_name Viettel Timor Leste   
0                                 0.0   
1                                 0.0   
2                                 0.0   
3                                 2.0   
4                                 2.0   
5                                 0.0   
6                                 0.0   
7                                 0.0   
8                                 0.0   
9                                11.0   
10                                0.0   
11                               28.0   
12                                0.0   
13                                0.0   
14                                4.0   

                                                                   \
probe_asn                                                AS150881   
probe_as_org_name INTERSTELLAR DIGITAL TECHNOLOGY COMPANY LIMITED   
0                                                             0.0   
1                                                             0.0   
2                                                             0.0   
3                                                             0.0   
4                                                             0.0   
5                                                             0.0   
6                                                             0.0   
7                                                             0.0   
8                                                             0.0   
9                                                             0.0   
10                                                            0.0   
11                                                            2.0   
12                                                            0.0   
13                                                            0.0   
14                                                            1.0   

                                                                     \
probe_asn                     AS18403             AS24086   AS45899   
probe_as_org_name FPT Telecom Company Viettel Corporation VNPT Corp   
0                              1466.0                 0.0       0.0   
1                                 0.0                 0.0       0.0   
2                                 0.0                 0.0       0.0   
3                               106.0                 0.0     288.0   
4                                92.0                 0.0       0.0   
5                                 0.0                 0.0       0.0   
6                               767.0                 0.0       0.0   
7                              1861.0                 0.0       0.0   
8                                 0.0          